<a href="https://colab.research.google.com/github/johno-source/vox-grn/blob/main/vox-grn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of vox-grn
The purpose of this file is to analyse the vox-grn dataset on huggingface.

In [6]:
import pandas as pd
import json
import requests


In [8]:

# create a dataframe using a generator
def gen_vox_grn():
  resp = requests.get('https://raw.githubusercontent.com/johno-source/vox-grn/main/data/vox-grn.json')
  vox_dict = json.loads(resp.text)
  for iso in vox_dict.keys():
    lang_df = pd.json_normalize(vox_dict[iso])
    lang_df['iso'] = iso
    yield lang_df

vox_df = pd.concat(gen_vox_grn())